In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
base_url = "http://llumar.co.kr/sub_support/faq.php?curr_page={}&list_count=20&sch_str=&sch_cate="

# 데이터 저장용 리스트
data = []

# 페이지 순회
page_num = 1
while True:
    url = base_url.format(page_num)
    response = requests.get(url)
    
    if response.status_code == 200:  # 페이지가 존재하는 경우
        response.encoding = 'euc-kr'  # 한글 깨짐 방지
        soup = BeautifulSoup(response.text, 'html.parser')

        # 테이블 행 추출
        rows = soup.select('tr')

        # 행 반복 처리
        valid_data_found = False
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 4:
                no = cols[0].text.strip()
                importance = cols[1].text.strip()
                question_field = cols[2].text.strip()
                question_tag = cols[3].find('a')
                question = question_tag.text.strip()
                link = question_tag['href']
                
                # 본문 내용 가져오기
                full_url = f"http://llumar.co.kr/sub_support/{link}"
                detail_response = requests.get(full_url)
                detail_response.encoding = 'euc-kr'
                detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
                content_div = detail_soup.find('div', class_='tx-content-container')
                if content_div:
                    content = ' '.join(content_div.stripped_strings)
                else:
                    content = None
                    
                data.append([no, importance, question_field, question, content])
                valid_data_found = True

        if not valid_data_found:
            break

        next_page_num = page_num + 1
        next_page_link = soup.find('a', href=lambda href: href and f'curr_page={next_page_num}' in href)
        if not next_page_link:
            break

        page_num += 1

# 데이터프레임 생성 및 출력
df = pd.DataFrame(data, columns=['No.', '중요도', '질문분야', '질문', '본문'])
df.to_csv("faq_scraping_results.csv", index=False, encoding='utf-8-sig')
print("Data saved to faq_scraping_results.csv")
print(df)


Data saved to faq_scraping_results.csv
   No.    중요도     질문분야                                                 질문  \
0   39   ★★★★  제품/시공관련  타업체에서는 열차단 측정기를 보여주며 자사제품이 루마보다 차단력이 우수하다고 홍보하...   
1   38   ★★★★  제품/시공관련          루마썬팅필름 정품으로 시공 후 창마다 로고가 다 들어가 있지 않는 이유는?   
2   37   ★★★★  제품/시공관련                 루마썬팅 후 창문에 있는 QR코드가 인식이 잘 안되는 이유는?   
3   36  ★★★★★     기타관련                            올바른 윈도우 필름을 고르는 10가지 요령   
4   35  ★★★★★  제품/시공관련  루마가 세계 판매1위 썬팅필름 전문 브랜드인데 다른 썬팅브랜드와 비교해서 어떤 점이...   
5   34  ★★★★★  제품/시공관련                            로고가 흐릿하게 나오는 이유는 무엇인가요?   
6   33  ★★★★★  제품/시공관련                      썬팅필름 시공 후 겨울철에는 어떤 관리요령이 있나요?   
7   32   ★★★★  제품/시공관련  루마썬팅의 완벽한 자외선 차단으로 인하여 인체에 필요한 비타민D의 생성에 방해가 되...   
8   31   ★★★★  제품/시공관련  집안내 세균을 살균하는 자외선을 루마필름이 99%이상 차단한다면 세균이 증식되는 결...   
9   30  ★★★★★  제품/시공관련                                   열차단효과 과장광고의 허와 실   
10  29  ★★★★★  제품/시공관련                         루마 컬러필름(그린/브라운) 무엇이 특별할까요?   
11  28  ★★★★★  제품/시공관련               

In [5]:
full_url

'http://llumar.co.kr/sub_support/faq.php?curr_page=2&list_count=20&sch_str=&sch_cate=&seq=12&mode=view'